In [1]:
import pandas as pd

<h4> Import data:

In [5]:
zz = pd.read_csv('CSV_files/中转.csv',header = None)
zz = zz.rename(columns = {0:'reviews'})
zz.head()

reviews
0  机场升级，服务降级。今天到T2找了半天中转休息室，走到三楼服务员说我的计分不够，只能提供金银...
1     今天来广州T2中转，服务员说现在不提供中转休息。我6个多小时。严重抗议！我们的权益被南航剥夺
2  页面字体可以稍微大一些。比如中转部分眼神儿不好，根本看不清楚。还有各项功能，也可以稍微大一些...
3  我今天乘坐南航从上海到旧金山的飞机CZ3544，需要在武汉中转。票号7821127333。上...
4  我从始至终选乘南方航空的飞机无论国内行程国外行程 无论票价高低 但是在累积里程的时候我发现 ...

<h4> 分词：

In [10]:
review_0 = zz.reviews.iloc[0]

In [11]:
import jieba
import jieba.posseg as pseg
import jieba.analyse

seg_list = jieba.cut(review_0, cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/_d/b3chzbkx5vgg1wtjm942qj4m0000gn/T/jieba.cache
Loading model cost 1.125 seconds.
Prefix dict has been built succesfully.


Default Mode: 机场/ 升级/ ，/ 服务/ 降级/ 。/ 今天/ 到/ T2/ 找/ 了/ 半天/ 中转/ 休息室/ ，/ 走到/ 三楼/ 服务员/ 说/ 我/ 的/ 计分/ 不够/ ，/ 只能/ 提供/ 金银/ 卡/ 会员/ 休息/ 。/ 然后/ 到/ 中转/ 服务/ 询问/ ，/ 回答/ 我/ 南航/ 转/ 到/ 丅/ 2/ 后/ 暂时/ 没有/ 这个/ 服务/ 。/ 想/ 问/ 一下/ 你们/ 可以/ 随便/ 取消/ 这项/ 服务/ 吗/ ？/ 是不是/ 机场/ 再/ 升级/ 一次/ ，/ 下次/ 就/ 取消/ 所有/ 的/ 座位/ 。


In [12]:
# 关键词提取
keywords_0 = jieba.analyse.extract_tags(review_0, topK=20, withWeight=True, allowPOS=('n','nr','ns'))
keywords_0

[('机场', 2.014140778422857),
 ('计分', 1.5508793163285712),
 ('降级', 1.3671349016699998),
 ('服务员', 1.2728921521714285),
 ('座位', 1.1338711869085716),
 ('会员', 1.0640898139342858)]

In [32]:
# 移除stop words 
# 直接采用open file as read 模式；区别于之前使用pandas来process.

# 创建停用词list  
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  

# 对句子进行分词  
def seg_sentence(sentence):  
    sentence_seged = jieba.cut(sentence.strip())  
    stopwords = stopwordslist('./stopwords.txt')  # 这里加载停用词的路径  
    outstr = ''  
    for word in sentence_seged:  
        if word not in stopwords:  
            if word != '\t':  
                outstr += word  
                outstr += " "  
    return outstr 

# handle tf-idf (Q: tf-idf based on single doc. rather than the whole corpus? 有意义吗？)
def get_topN_tf_idf(content,topK=20):
    tags = jieba.analyse.extract_tags(content, topK)
    print(" ".join(tags))
    
inputs = open('./CSV_files/中转.csv', 'r', encoding='utf-8')  
outputs = open('./CSV_files/中转output.txt', 'w')  
for line in inputs:  
    line_seg = seg_sentence(line)  # 这里的返回值是字符串  
#     line_seg = get_topN_tf_idf（line_seg,topK=20)   # 移除低频词
    outputs.write(line_seg + '\n')  
outputs.close()  
inputs.close() 

﻿ 机场 升级 服务 降级 T2 找 半天 中转 休息室 走到 三楼 服务员 说 计分 只能 提供 金银 卡 会员 休息 中转 服务 询问 回答 南航 转 丅 暂时 服务 想 问 随便 取消 这项 服务 机场 升级 下次 取消 座位 
广州 T2 中转 服务员 说 提供 中转 休息 小时 抗议 权益 南航 剥夺 
页面 字体 稍微 中转 眼神儿 不好 看不清楚 各项 功能 稍微 字体 至少 选择 大字体 年纪 眼睛 花 看不见 
乘坐 南航 上海 旧金山 飞机 CZ3544 武汉 中转 票号 7821127333 上海 柜台 服务 人员 说 发行 李到 目的地 武汉 提 行李 时 发现 全新 行李箱 摔坏 找 不到 南航 服务 人员 帮 办理 中转 住宿 机场 行李 寄存处 关闭 客服 武汉 中转 柜台 电话 两个 电话 没人接 听 说 电话 关机 行李 很难 拖 摔坏 行李箱 走动 晚班 飞机 晚班 服务 
从始至终 选乘 南方 航空 飞机 国内 行程 国外 行程   票价   累积 里程 发现   行程 大部分 墨尔本 广州 中转   里程 100% 累积   每次 携程 订机票   并不知道 仓位 累积 里程   经济舱 仓位 之分   不合理   飞行 距离 万公里   希望 回复   南方 航空 失望   同期 航班 航空公司 价格 便宜   里程 百分百 累积 
乘坐 11 月 19 日 21 20 济南 广州 CZ3708 航班 中转 广州 乘坐 次日 11 月 20 日 CZ8315 航班 到达 越南 河内 内排 机场 票号 7843117208881 行李 济南 到达 越南 河内 到达 河内 机场 行李 提取 处后 发现 行李箱 损坏 河内 机场 未找到 南航 值班 处 特 申请 贵司 赔付 行李箱 一件 

相关 行李箱 损坏 照片 视频 保留 上传 望贵司 答复 

谢谢 
白云 办理 中转 住宿 柜台 服务态度 真的 一句 话 不愿 讲 全程 两句话 选 一家 酒店 kfc 班车 等待 久 
太差 太差 太差 南航 真差 无锡 买 通程票 张家界 长沙 中转 座 南航 CZ3164 正规 航空公司 无锡 机场 行程 答复 出发 机场 长沙 答复 打要 找 订票 平台 邮寄 增加 20 快递费 正规 公司 不坑 爹 公司 
两段 南航 中转 济

In [23]:
def get_output_file(inputs_path,outputs_path):
    '''
    This function inputs the source data files, 
    and return the 分词 file as ouput.    
    '''
    
    def stopwordslist(filepath):  
        stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
        return stopwords  

    # 对句子进行分词  
    def seg_sentence(sentence):  
        sentence_seged = jieba.cut(sentence.strip())  
        stopwords = stopwordslist('./stopwords.txt')  # 这里加载停用词的路径  
        outstr = ''  
        for word in sentence_seged:  
            if word not in stopwords:  
                if word != '\t':  
                    outstr += word  
                    outstr += " "  
        return outstr 

    # handle tf-idf (Q: tf-idf based on single doc. rather than the whole corpus? 有意义吗？)
    def get_topN_tf_idf(content,topK=20):
        tags = jieba.analyse.extract_tags(content, topK)
        print(" ".join(tags))
    
    inputs = open(inputs_path, 'r', encoding='utf-8')  
    outputs = open(outputs_path, 'w')  
    for line in inputs:  
        line_seg = seg_sentence(line)  # 这里的返回值是字符串  
#         line_seg = get_topN_tf_idf（line_seg,topK=20)  # 移除低频词
        outputs.write(line_seg + '\n')  
    outputs.close()  
    inputs.close() 


categories = ['中转','出发','到达','售后','性能','机上','行程管理','计划','设计','预订']
    
for cat in categories:
    input_path = './CSV_files/' + cat + '.csv'
    output_path = './output_data/' + cat + 'output.txt'
    get_output_file(input_path, output_path)

<h4> TODO: 建立向量空间模型(特征工程）：

TODO: train, test split dataset

TODO: word2vec (法1)
TODO: doc2vec  (法2)
TODO: LDA   (法3)

In [ ]:
# TODO: data preprocessing 
'''
combine
'''

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

<h4> TODO: 使用neural network:

使用neural network可以免去特征工程

In [ ]:
# CNN (法4) -> textCNN
# RNN (法5) -> textRNN